# Prática Independente: A maldição da dimensionalidade.

#### Para entender o efeito da quantidade de dimensões na capacidade preditiva do modelo, vamos trabalhar com um classificador de notícias de dois jornais argentinos para tentar distingui-los com base no vocabulário que eles usam. <br />

#### Para isso, vamos implementar um modelo de tipo `Naïve Bayes` com vetorização de tipo `TF-IDF`.

#### Exercício 1: Carregue o arquivo `clarin.csv` em um dataframe, definindo para ele um atributo de classe `class = 0`. Carregue também o arquivo 'pagina12.csv’ em um dataframe separado, também criando um atributo de classe `class = 1`. Faça a concatenação dos dois dataframes.

#### Realize uma análise exploratória, com uma limpeza dos dados, estudando a existência de dados nulos e fazendo a imputação de dados onde necessário, escolha o método que mais lhe convir.

#### Remova as seções em que os dois jornais usam um vocabulário semelhante e de domínio muito específico, como esportes, por exemplo.

#### Construa uma Corpus de documentos composto de uma coluna sobre a que iremos prever concatenando o título, resumo e corpo das diferentes notícias.

#### Realize a normalização dos dados, convertendo o corpus de notícias para minúsculas, removendo acentos que possam gerar diferenças artificiais entre as palavras, removendo também a pontuação, deixando apenas palavras.

### 1. Limpeza

#### 1.1 Faltantes

A partir do dataset observamos que os campos que provavelmente contêm vocabulário relevante são "corpo", "título" e "resumo".
Remover da análise os registros que não possuem corpo ou título disponíveis e completar os resumos faltantes com um campo em branco


#### 1.2 Suplementos relevantes

Para melhorar a classificação, é conveniente retirar as seções em que os dois jornais usam um vocabulário semelhante e de domínio muito específico como, por exemplo, aqueles relacionados a esportes.

#### 1.3 Corpus

Construa a coluna sobre a que iremos prever concatenando o título, resumo e corpo das diferentes notícias.

#### 1.4 Normalização do texto

Primeiro, é conveniente passar todo o corpus a letras minúsculas.

Para melhorar o classificador, é importante retirar todos os acentos que possam gerar diferenças artificiais entre as palavras, caso estejam colocados ou não. Hint: Usar o módulo unidecode

Uma vez retirados os acentos, remover todos os sinais de pontuação para deixar somente palavras. Dica: usar uma expressão regular, por exemplo, r'([^\s\w]|_)+' para substituir tudo que não for palavra.

#### Exercício 2: Vetorize o corpus resultante com a técnica `TF-IDF` e aplique um modelo `Naive Bayes` com um `split` simples entre os subconjuntos de treino e teste. 

#### Discuta qual é a dimensão da matriz de atributos e a precisão obtida. Plote também uma matriz de confusão.

#### Exercício 3: Realize a redução da dimensionalidade do modelo apresentado.

#### Considerando a relação entre o número de atributos e o número de casos de treinamento, discuta se, de acordo com a maldição da dimensionalidade, esta relação é um problema.

#### Faça uso da lista de `stopwords `, com palavras do idioma espanhol que não têm um peso semântico significativo para a redução da dimensionalidade. Treinem novamente o modelo, removendo as `stopwords` contidas no arquivo "stopwords.csv".

#### Discuta o que acontece com o número de dimensões da matriz de atributos e o que acontece com sua precisão.

#### Exercício 4: Seguindo adiante na redução de dimensões, e considerando que modelo melhora quando removemos as `stopwords`, mas que ainda temos um número alto de dimensões, dada a quantidade de dados que temos. Considere:

#### Para remover da análise as palavras que aparecem menos do que determinado número de vezes, a classe `TfidfVectorizer` tem um parâmetro `min_df`, que criar o vocabulário, ignora termos que tenham uma frequência de documento estritamente menor que o limite especificado.

#### Definir o parâmetro `min_df = 6` e volte a executar o modelo. Discuta quanto se reduz a dimensão da matriz de atributos e quanto o desempenho do algoritmo é melhorado com a adoção desse procedimento.